# Volume 1, Chapter 2: Introduction to LLMs

**Token Calculator & Model Cost Comparison**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/main/Volume-1-Foundations/Colab-Notebooks/Vol1_Ch2_Token_Calculator.ipynb)

## What You'll Build

By the end of this notebook, you'll have:
- A working token calculator for network configs
- Cost comparison tool across all major models
- Understanding of how your network data tokenizes
- Batch processing cost projections

## Prerequisites

- Completed Chapter 1
- Python basics
- No API key needed for token counting (only for actual API calls)

**Estimated Time**: 30-45 minutes  
**Cost**: Free (token counting only, no API calls required)

---
## 🔧 Setup: Install Dependencies

We'll use `tiktoken` - OpenAI's tokenizer library. It's a good approximation for all models (Claude, GPT, etc.).

In [ ]:
# Install tiktoken
!pip install -q tiktoken

print("✓ tiktoken installed successfully!")

---
## 📊 Current Model Pricing (January 2026)

Let's define the pricing for all major models. This will be used throughout the notebook.

In [ ]:
# Pricing per 1M tokens (as of January 2026)
# Check https://anthropic.com/pricing and https://openai.com/api/pricing for latest
PRICING = {
    "claude-haiku-4.5": {
        "input": 1.00,
        "output": 5.00,
        "context": 200_000,
        "name": "Claude Haiku 4.5"
    },
    "claude-sonnet-4.5": {
        "input": 3.00,
        "output": 15.00,
        "context": 200_000,
        "name": "Claude Sonnet 4.5"
    },
    "claude-opus-4.5": {
        "input": 5.00,
        "output": 25.00,
        "context": 200_000,
        "name": "Claude Opus 4.5"
    },
    "gpt-4o-mini": {
        "input": 0.15,
        "output": 0.60,
        "context": 128_000,
        "name": "GPT-4o-mini"
    },
    "gpt-4o": {
        "input": 2.50,
        "output": 10.00,
        "context": 128_000,
        "name": "GPT-4o"
    },
    "gemini-1.5-pro": {
        "input": 1.25,
        "output": 5.00,
        "context": 2_000_000,
        "name": "Gemini 1.5 Pro"
    }
}

# Display pricing table
print("=" * 80)
print("CURRENT MODEL PRICING (January 2026)")
print("=" * 80)
print(f"{'Model':<20} {'Input ($/1M)':<15} {'Output ($/1M)':<15} {'Context'}")
print("-" * 80)
for model_id, info in PRICING.items():
    print(f"{info['name']:<20} ${info['input']:<14.2f} ${info['output']:<14.2f} {info['context']:,}")
print("=" * 80)

---
## 🔢 Demo 1: Basic Token Counting

Let's start simple - count tokens in different types of text.

In [ ]:
import tiktoken

def count_tokens(text: str) -> int:
    """Count tokens using GPT-4's tokenizer (good approximation for all models)."""
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

def show_tokens(text: str, description: str = ""):
    """Display text with token count and breakdown."""
    encoding = tiktoken.get_encoding("cl100k_base")
    tokens = encoding.encode(text)
    token_strings = [encoding.decode([token]) for token in tokens]
    
    print(f"\n{'='*60}")
    if description:
        print(f"Description: {description}")
    print(f"Text: {text}")
    print(f"Tokens: {token_strings}")
    print(f"Token count: {len(tokens)}")
    print(f"Characters: {len(text)}")
    print(f"Ratio: {len(text)/len(tokens):.2f} chars/token")
    print(f"{'='*60}")

# Examples from Chapter 2
examples = [
    ("Hello world", "Simple English"),
    ("interface GigabitEthernet0/0", "Cisco interface command"),
    ("int Gi0/0", "Abbreviated version"),
    ("192.168.1.1", "IP address"),
    ("255.255.255.0", "Subnet mask"),
    ("router bgp 65001", "BGP command"),
]

for text, desc in examples:
    show_tokens(text, desc)

### 💡 Key Observations

Notice:
- Simple English: ~2-3 chars/token
- Network configs: ~2-4 chars/token
- IP addresses: Each dot (.) is a separate token
- Abbreviated commands use fewer tokens ("int Gi0/0" vs "interface GigabitEthernet0/0")

---
## 💰 Demo 2: Cost Calculator

Now let's calculate costs across all models.

In [ ]:
def calculate_cost(input_tokens: int, output_tokens: int, model: str) -> dict:
    """
    Calculate cost for a given model.
    
    Returns dict with:
    - input_cost: Cost of input tokens
    - output_cost: Cost of output tokens
    - total_cost: Total cost
    - fits: Whether it fits in context window
    """
    if model not in PRICING:
        raise ValueError(f"Unknown model: {model}")
    
    pricing = PRICING[model]
    
    input_cost = (input_tokens / 1_000_000) * pricing["input"]
    output_cost = (output_tokens / 1_000_000) * pricing["output"]
    total_cost = input_cost + output_cost
    
    total_tokens = input_tokens + output_tokens
    fits = total_tokens < pricing["context"]
    
    return {
        "input_cost": input_cost,
        "output_cost": output_cost,
        "total_cost": total_cost,
        "fits": fits,
        "model_name": pricing["name"],
        "context_window": pricing["context"]
    }

# Example: 10,000-line router config
input_tokens = 13_000  # ~10K line config + prompt
output_tokens = 2_000  # Detailed analysis

print("\n" + "="*80)
print(f"COST ANALYSIS: {input_tokens:,} input tokens + {output_tokens:,} output tokens")
print("="*80)
print(f"{'Model':<20} {'Input':<10} {'Output':<10} {'Total':<10} {'Fits?'}")
print("-"*80)

for model_id in PRICING.keys():
    result = calculate_cost(input_tokens, output_tokens, model_id)
    fits_icon = "✅" if result["fits"] else "❌"
    print(f"{result['model_name']:<20} ${result['input_cost']:.4f}   ${result['output_cost']:.4f}   ${result['total_cost']:.4f}   {fits_icon}")

print("="*80)

### 💡 Key Insight

Notice:
- **Output tokens cost 3-5x more** than input tokens
- GPT-4o-mini is **20x cheaper** than Claude Sonnet
- All models fit this config in their context window

---
## 📁 Demo 3: Analyze Network Configs

Let's create sample network configs and analyze their token counts.

In [ ]:
# Sample configs of different sizes
SMALL_CONFIG = """!
hostname Branch-Router-01
!
interface GigabitEthernet0/0
 ip address 192.168.1.1 255.255.255.0
 no shutdown
!
router ospf 1
 network 192.168.1.0 0.0.0.255 area 0
!
end
"""

MEDIUM_CONFIG = SMALL_CONFIG * 50  # Simulate 50 interfaces
LARGE_CONFIG = SMALL_CONFIG * 500   # Simulate 500 interfaces

def analyze_config(config: str, config_name: str, expected_output: int = 2000):
    """
    Analyze a config and show token counts and costs across all models.
    """
    tokens = count_tokens(config)
    lines = len(config.split('\n'))
    
    print(f"\n{'='*80}")
    print(f"CONFIG ANALYSIS: {config_name}")
    print("="*80)
    print(f"Lines: {lines:,}")
    print(f"Characters: {len(config):,}")
    print(f"Tokens: {tokens:,}")
    print(f"Chars/token: {len(config)/tokens:.2f}")
    print()
    print(f"Cost estimates (assuming {expected_output:,} output tokens):")
    print("-"*80)
    print(f"{'Model':<20} {'Cost':<12} {'Monthly (100x)':<15} {'Fits?'}")
    print("-"*80)
    
    for model_id in PRICING.keys():
        result = calculate_cost(tokens, expected_output, model_id)
        monthly = result['total_cost'] * 100
        fits_icon = "✅" if result["fits"] else "❌"
        print(f"{result['model_name']:<20} ${result['total_cost']:.4f}    ${monthly:.2f}          {fits_icon}")
    
    print("="*80)

# Analyze all three configs
analyze_config(SMALL_CONFIG, "Small Config (10 lines)")
analyze_config(MEDIUM_CONFIG, "Medium Config (500 lines)")
analyze_config(LARGE_CONFIG, "Large Config (5,000 lines)")

---
## 🎯 Lab 1: Interactive Token Counter

**Your turn!** Paste your own configs or commands and see how they tokenize.

In [ ]:
# Interactive token counter
your_text = input("Paste your config/command (or press Enter for example): ")

if not your_text.strip():
    your_text = "interface GigabitEthernet0/0\n ip address 10.0.0.1 255.255.255.0"
    print(f"Using example: {your_text[:50]}...")

show_tokens(your_text, "Your Input")

# Calculate cost
tokens = count_tokens(your_text)
print("\nCost to analyze (assuming 2,000 token response):")
print("-"*60)

for model_id in ["gpt-4o-mini", "claude-haiku-4.5", "claude-sonnet-4.5"]:
    result = calculate_cost(tokens, 2000, model_id)
    print(f"{result['model_name']:<20} ${result['total_cost']:.4f}")

---
## 📊 Demo 4: Context Window Testing

Let's see which models can handle large configs.

In [ ]:
# Test different config sizes
test_sizes = [
    (1_000, "Small (1K lines)"),
    (10_000, "Medium (10K lines)"),
    (50_000, "Large (50K lines)"),
    (100_000, "Very Large (100K lines)"),
    (500_000, "Massive (500K lines)"),
]

# Estimate tokens (assuming ~4 chars per line, ~2.5 tokens per 4 chars)
def estimate_tokens_from_lines(lines: int) -> int:
    return int(lines * 50 / 4)  # ~50 chars per line, ~4 chars per token

print("\n" + "="*80)
print("CONTEXT WINDOW FIT TEST")
print("="*80)
print(f"{'Config Size':<20} {'Tokens':<12} {'GPT-4o':<10} {'Claude':<10} {'Gemini'}")
print("-"*80)

for lines, desc in test_sizes:
    tokens = estimate_tokens_from_lines(lines)
    
    gpt_fit = "✅" if tokens < 128_000 else "❌"
    claude_fit = "✅" if tokens < 200_000 else "❌"
    gemini_fit = "✅" if tokens < 2_000_000 else "❌"
    
    print(f"{desc:<20} {tokens:>10,}  {gpt_fit:<10} {claude_fit:<10} {gemini_fit}")

print("="*80)
print("\nKey Insights:")
print("- GPT-4o/4o-mini: Max ~100K lines (128K tokens)")
print("- Claude (all): Max ~160K lines (200K tokens)")
print("- Gemini 1.5 Pro: Max ~1.6M lines (2M tokens)")

---
## 💵 Demo 5: Batch Processing Costs

Calculate monthly costs for analyzing your entire network.

In [ ]:
def calculate_batch_costs(device_count: int, avg_tokens: int, analyses_per_month: int):
    """
    Calculate monthly costs for batch processing.
    """
    total_analyses = device_count * analyses_per_month
    output_tokens = 2000  # Assume 2K token response per analysis
    
    print(f"\n{'='*80}")
    print(f"BATCH COST PROJECTION")
    print("="*80)
    print(f"Devices: {device_count:,}")
    print(f"Avg tokens per config: {avg_tokens:,}")
    print(f"Analyses per device per month: {analyses_per_month}")
    print(f"Total analyses per month: {total_analyses:,}")
    print()
    print(f"{'Model':<20} {'Per Analysis':<15} {'Monthly':<15} {'Annual'}")
    print("-"*80)
    
    for model_id in PRICING.keys():
        result = calculate_cost(avg_tokens, output_tokens, model_id)
        monthly = result['total_cost'] * total_analyses
        annual = monthly * 12
        
        print(f"{result['model_name']:<20} ${result['total_cost']:.4f}        ${monthly:>8.2f}      ${annual:>10.2f}")
    
    print("="*80)

# Example: 1,000 devices, analyze weekly
calculate_batch_costs(
    device_count=1000,
    avg_tokens=12_000,  # ~10K line config
    analyses_per_month=4  # Weekly
)

# Your network - customize these values!
print("\n" + "="*80)
print("CALCULATE FOR YOUR NETWORK:")
print("="*80)

try:
    your_devices = int(input("How many devices? (default 100): ") or "100")
    your_tokens = int(input("Avg tokens per config? (default 10000): ") or "10000")
    your_frequency = int(input("Analyses per device per month? (default 4): ") or "4")
    
    calculate_batch_costs(your_devices, your_tokens, your_frequency)
except:
    print("Using default values...")
    calculate_batch_costs(100, 10000, 4)

---
## 🎯 Lab 2: Model Comparison for Your Use Case

Compare models based on YOUR specific requirements.

In [ ]:
# Model selection helper
def recommend_model(tokens: int, task_type: str, budget: str):
    """
    Recommend the best model based on requirements.
    
    task_type: 'simple', 'analysis', 'critical'
    budget: 'low', 'medium', 'high'
    """
    recommendations = {
        'simple': {
            'low': 'gpt-4o-mini',
            'medium': 'claude-haiku-4.5',
            'high': 'claude-sonnet-4.5'
        },
        'analysis': {
            'low': 'claude-haiku-4.5',
            'medium': 'claude-sonnet-4.5',
            'high': 'claude-sonnet-4.5'
        },
        'critical': {
            'low': 'claude-sonnet-4.5',
            'medium': 'claude-sonnet-4.5',
            'high': 'claude-opus-4.5'
        }
    }
    
    model_id = recommendations[task_type][budget]
    
    # Check if it fits
    context_limit = PRICING[model_id]['context']
    fits = tokens < (context_limit * 0.9)  # 90% safety margin
    
    print(f"\n{'='*80}")
    print(f"MODEL RECOMMENDATION")
    print("="*80)
    print(f"Your requirements:")
    print(f"  - Task type: {task_type}")
    print(f"  - Budget: {budget}")
    print(f"  - Input tokens: {tokens:,}")
    print()
    print(f"Recommended: {PRICING[model_id]['name']}")
    print(f"  - Context window: {context_limit:,} tokens")
    print(f"  - Will it fit? {'✅ Yes' if fits else '❌ No - Consider chunking or Gemini'}")
    print()
    
    # Show cost
    result = calculate_cost(tokens, 2000, model_id)
    print(f"Cost per analysis: ${result['total_cost']:.4f}")
    print(f"Cost for 1,000 analyses: ${result['total_cost'] * 1000:.2f}")
    print("="*80)
    
    return model_id

# Examples
print("\nExample 1: Simple syntax check")
recommend_model(tokens=5000, task_type='simple', budget='low')

print("\nExample 2: Security analysis")
recommend_model(tokens=15000, task_type='analysis', budget='medium')

print("\nExample 3: Critical audit")
recommend_model(tokens=20000, task_type='critical', budget='high')

---
## 📝 Demo 6: Complete Token Calculator

The complete production-ready tool with all features.

In [ ]:
#!/usr/bin/env python3
"""
Complete Token Calculator for Network Engineers
Production-ready implementation
"""

import tiktoken
from typing import Dict, List

class TokenCalculator:
    def __init__(self):
        self.encoding = tiktoken.get_encoding("cl100k_base")
        self.pricing = PRICING
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text."""
        return len(self.encoding.encode(text))
    
    def analyze(self, text: str, expected_output: int = 2000) -> Dict:
        """
        Complete analysis of text with costs for all models.
        """
        input_tokens = self.count_tokens(text)
        
        results = {
            'input_tokens': input_tokens,
            'output_tokens': expected_output,
            'characters': len(text),
            'lines': len(text.split('\n')),
            'models': {}
        }
        
        for model_id, pricing in self.pricing.items():
            input_cost = (input_tokens / 1_000_000) * pricing['input']
            output_cost = (expected_output / 1_000_000) * pricing['output']
            total_cost = input_cost + output_cost
            
            fits = (input_tokens + expected_output) < pricing['context']
            
            results['models'][model_id] = {
                'name': pricing['name'],
                'input_cost': input_cost,
                'output_cost': output_cost,
                'total_cost': total_cost,
                'fits': fits,
                'context': pricing['context']
            }
        
        return results
    
    def format_report(self, results: Dict) -> str:
        """Format results as a readable report."""
        report = []
        report.append("="*80)
        report.append("TOKEN ANALYSIS REPORT")
        report.append("="*80)
        report.append(f"Input: {results['input_tokens']:,} tokens ({results['characters']:,} chars, {results['lines']:,} lines)")
        report.append(f"Expected output: {results['output_tokens']:,} tokens")
        report.append("")
        report.append(f"{'Model':<20} {'Cost':<12} {'100x':<12} {'1000x':<12} {'Fits?'}")
        report.append("-"*80)
        
        # Sort by cost
        sorted_models = sorted(
            results['models'].items(),
            key=lambda x: x[1]['total_cost']
        )
        
        for model_id, data in sorted_models:
            cost_100 = data['total_cost'] * 100
            cost_1000 = data['total_cost'] * 1000
            fits_icon = "✅" if data['fits'] else "❌"
            
            report.append(
                f"{data['name']:<20} ${data['total_cost']:.4f}    "
                f"${cost_100:>8.2f}   ${cost_1000:>8.2f}    {fits_icon}"
            )
        
        report.append("="*80)
        return "\n".join(report)

# Test the calculator
calc = TokenCalculator()

# Analyze a sample config
test_config = """hostname Core-Router-01
interface GigabitEthernet0/0
 ip address 10.0.1.1 255.255.255.252
 ip ospf cost 10
!
router ospf 1
 network 10.0.0.0 0.0.255.255 area 0
!
router bgp 65001
 neighbor 10.0.1.2 remote-as 65002
!
""" * 100  # Simulate larger config

results = calc.analyze(test_config)
print(calc.format_report(results))

---
## 🎓 Self-Assessment

Can you answer these questions?

1. **How many tokens is the IP address "192.168.1.1"?**
   <details>
   <summary>Answer</summary>
   7 tokens: ["192", ".", "168", ".", "1", ".", "1"]
   </details>

2. **Which costs more: input tokens or output tokens?**
   <details>
   <summary>Answer</summary>
   Output tokens cost 3-5x more than input tokens across all models.
   </details>

3. **What's the cheapest model for simple tasks?**
   <details>
   <summary>Answer</summary>
   GPT-4o-mini at $0.15/$0.60 per million tokens.
   </details>

4. **Which model has the largest context window?**
   <details>
   <summary>Answer</summary>
   Gemini 1.5 Pro with 2M tokens (10x larger than GPT-4o/Claude).
   </details>

5. **If a 10K-line config costs $0.069 with Sonnet, what's the monthly cost for 1,000 devices analyzed weekly?**
   <details>
   <summary>Answer</summary>
   1,000 devices × 4 weeks × $0.069 = $276/month
   </details>

---
## 🎉 Congratulations!

You now have:

✅ A working token calculator for any network config  
✅ Cost comparison across all major models  
✅ Understanding of how network data tokenizes  
✅ Batch processing cost projections  
✅ Model selection framework  

## 💾 Save Your Work

Download this notebook (File → Download → Download .ipynb) and save it to your toolkit!

## 🚀 Next Steps

**Chapter 3: Choosing the Right Model**
- Benchmark models on your specific workloads
- Understand quality vs cost tradeoffs
- Build a model selection decision tree

---

**Questions?** Open an issue on [GitHub](https://github.com/eduardd76/AI_for_networking_and_security_engineers/issues)

**Want more?** Check out the full book at [vExpertAI.com](https://vexpertai.com)